In [1]:
import pandas as pd
import logging
import ast
import numpy as np
from sentence_transformers import SentenceTransformer
from typing import Dict
import re

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import google.generativeai as genai
import os

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

llm =  genai.GenerativeModel("gemini-2.0-flash")


In [19]:
import chromadb

chroma_client = chromadb.PersistentClient(path="chroma_db_v1")
collection = chroma_client.get_or_create_collection(name="new_documents")

In [5]:
EMBEDDING_MODEL =  'paraphrase-multilingual-mpnet-base-v2'

In [21]:
def retriever(question):
    model = SentenceTransformer(EMBEDDING_MODEL)
    query_embedding = model.encode([question], convert_to_tensor=True).tolist()

    results = collection.query(
        query_embeddings=query_embedding,
        n_results=3
    )
    return results

def format_docs(docs):
    return "\n\n".join(['\n'.join([res['name'],res["description"]]) for res in docs["metadatas"][0]])
# Example query
docs = retriever("Nốt hương của Bleu Channel là gì?")

print("Generated Response:", format_docs(docs))

Generated Response: Scandal Pour Homme
Hương Đầu: Cây Xô Thơm, Cam quýt Hương Giữa: Caramel, Đậu Tonka Hương Cuối: Cỏ Hương Bài. Năm 2021, thương hiệu đình đám Jean Paul Gaultier trở lại cùng với một siêu phẩm mới với cái tên quen thuộc "SCANDAL", nhưng lần này là phiên bản dành cho các chàng trai. Mùi hương đầy hứa hẹn này chưa gì đã đạt được mục đích làm choáng ngợp những tín đồ nước hoa với ngoại hình chiếc vương miện đầy mạnh mẽ, được lấy cảm hứng từ những vị đấu sĩ quyền anh kiêu hãnh và oai phong. Miêu tả tổng quan về mùi hương thì Scandal Pour Homme thuộc tông mùi Ngọt - một nét đặc trưng, không lẫn vào đâu được của thương hiệu Jean Paul Gaultier. Scandal Pour Homme mở ra với mùi chua nhẹ nhàng và tươi quát của những quả Cam Quýt mọng nước cùng một ít màu xanh lục, tựa như thảo mộc của nốt Xô Thơm. Khá chóng vánh, hai nốt hương ấy dần nhường chỗ cho ngôi sao chính, một tổ hợp đã tạo nên vị ngọt béo bở, đầy lôi cuốn và quyến rũ đến từ Caramel và đậu Tonka. Ngọt nhưng vẫn thể hiện

In [7]:
class RunnableChain:
    """Lớp bọc lambda function để hỗ trợ .invoke()"""
    def __init__(self, func):
        self.func = func

    def invoke(self, **kwargs):
        return self.func(**kwargs)

    def __or__(self, other):
        """Hỗ trợ chaining nhiều bước"""
        if not callable(other):
            raise TypeError(f"Cannot chain with type {type(other)}")

        # Ensure the next step is also a RunnableChain
        return RunnableChain(lambda **kwargs: other.invoke(self.func(**kwargs)))


class ChatPromptTemplate:
    def __init__(self, template: str):
        self.template = template
        self.variables = self.extract_variables(template)

    def extract_variables(self, template: str):
        return re.findall(r"\{(\w+)\}", template)

    def format(self, **kwargs) -> str:
        """Điền giá trị vào template"""
        missing_vars = [var for var in self.variables if var not in kwargs]
        if missing_vars:
            raise ValueError(f"Missing variables: {missing_vars}")

        return self.template.format(**kwargs)

    @classmethod
    def from_template(cls, template: str):
        return cls(template)

    def __or__(self, model):
        if not hasattr(model, "generate_content"):
            raise TypeError(f"Cannot chain with type {type(model)}")

        return RunnableChain(lambda **kwargs: model.generate_content(self.format(**kwargs)))

In [9]:
class RunnableFunction:
    def __init__(self, func):
        self.func = func  

    def __or__(self, other):
        if not callable(other):
            raise TypeError(f"Cannot chain with type {type(other)}")
        return RunnableFunction(lambda x: other(self.func(x)))

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

    def invoke(self, x):
        return self.func(x)


def str_output_parser(output):
    if hasattr(output, "text"):
        return output.text  # Nếu output có thuộc tính `.text`
    if isinstance(output, dict) and "text" in output:
        return output["text"]  # Nếu output là dict chứa key "text"
    return str(output)  # Chuyển tất cả về string


In [22]:
template = """Using the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Khởi tạo parser
parser = RunnableFunction(str_output_parser)

# Xây dựng chain
chain = (
    prompt 
    | llm 
    | parser
)

question="Nốt hương của Bleu Chanel là gì?"

docs = retriever(question)

# Chạy pipeline
response = chain.invoke(context=format_docs(docs), question=question)
print(response)


Dựa trên đoạn văn bản bạn cung cấp, có hai loại Bleu de Chanel được mô tả:

**Bleu de Chanel Parfum:**

*   **Hương Đầu:** Vỏ chanh, Cam Bergamot, Cây bạc hà
*   **Hương giữa:** Hoa Oải Hương, Hoa phong lữ, Quả dứa (quả thơm)
*   **Hương cuối:** Gỗ tuyết tùng, Gỗ đàn hương, Đậu Tonka

**Bleu de Chanel (phiên bản gốc):**

*   **Hương đầu:** Quả bưởi, Quả chanh vàng, Bạc hà, Tiêu hồng.
*   **Hương giữa:** Gừng, Nhục đậu khấu, Hoa nhài, Iso E Super.
*   **Hương cuối:** Nhang, Cỏ hương bài, Gỗ tuyết tùng, Gỗ đàn hương, Hoắc hương, Nhựa hương Labdanum, Xạ hương trắng.

Vậy nên, nốt hương của Bleu de Chanel sẽ khác nhau tùy thuộc vào phiên bản Parfum hay phiên bản gốc.



# Khám phá sở thích, nhu cầu của khách hàng.


## Tạo cuộc hội thoại với khách hàng

In [ ]:
prompt_begin_template = """
Bạn là trợ lý ảo của Namperfume, chuyên tư vấn nước hoa. Hãy bắt đầu cuộc trò chuyện một cách tự nhiên và đặt câu hỏi một cách linh hoạt để hiểu rõ sở thích của khách hàng. Dưới đây là một số gợi ý về thông tin bạn có thể thu thập:

* Mùi hương khách hàng yêu thích là gì? (Ví dụ: tươi mát, hoa cỏ, gỗ, trái cây, vani,...)
* Xác định giới tính của khách hàng sử dụng? Mua cho bản thân hay mua tặng cho ai?
* Khách hàng muốn sử dụng nước hoa cho mục đích gì? (Hàng ngày, dịp đặc biệt, đi làm, hẹn hò,...)
* Phong cách mà khách hàng hướng đến là gì? (Năng động, thanh lịch, lãng mạn, cá tính, quyến rũ,...)
* Khách hàng mong muốn nước hoa mang lại cảm giác gì? (Tự tin, thư giãn, nổi bật, ấm áp,...)
* Thời tiết hoặc mùa nào mà khách hàng thường sử dụng nước hoa?
* Ngân sách mà khách hàng dự định chi cho nước hoa là bao nhiêu?
* Khách hàng có thương hiệu nước hoa yêu thích nào không?

**Lưu ý quan trọng:**

* Không nhất thiết phải hỏi tất cả các câu hỏi. Hãy lắng nghe câu trả lời của khách hàng và đặt câu hỏi tiếp theo một cách tự nhiên dựa trên thông tin đã có.
* Nếu bạn cảm thấy đã thu thập đủ thông tin để đưa ra gợi ý phù hợp, hãy chuyển sang bước giới thiệu sản phẩm mà không cần hỏi thêm.
* Ưu tiên tạo ra một cuộc trò chuyện thoải mái và hữu ích cho khách hàng.
* Chỉ hỏi chưa tư vấn sản phẩm cụ thể nào.
* Sau khi thu thập đủ thông tin thì nói khách hàng đợi để tìm kiếm sản phẩm phù hợp

######
ChatBot: Chào bạn, mình là trợ lý ảo của Namperfume. Rất vui được hỗ trợ bạn tìm kiếm mùi hương ưng ý. Bạn có thể cho mình biết bạn đang tìm kiếm loại nước hoa như thế nào không?

Khách hàng: {question}

ChatBot:
"""


prompt_begin = ChatPromptTemplate.from_template(prompt_begin_template)
input_prompt_begin = prompt_begin.format(question="Tôi muốn được tư vấn nước hoa")

response = llm.generate_content(input_prompt_begin)
print(f"ChatBot: {str_output_parser(response)}\n")

history = [input_prompt_begin]  # Danh sách lưu lịch sử hội thoại

while True:
    user_input = input("Bạn: ")
    

    if "exit" in user_input.lower():
        break
    
    print(f"Khách hàng: {user_input}\n")

    history.append(f"Khách hàng: {user_input}")
    full_prompt = "\n".join(history)
    
    # Chuỗi hội thoại đầy đủ
    response = llm.generate_content(full_prompt)

    # Xử lý phản hồi
    if hasattr(response, "text"):
        bot_response = response.text
    elif isinstance(response, dict) and "text" in response:
        bot_response = response["text"]
    else:
        bot_response = str(response)

    print(f"{bot_response}\n")
    # Lưu tin nhắn của chatbot vào lịch sử
    history.append(f"{bot_response}")



ChatBot: Tuyệt vời! Để giúp bạn tìm được mùi hương hoàn hảo nhất, bạn có thể chia sẻ một chút về sở thích của mình được không? Ví dụ, bạn thường thích những mùi hương như thế nào? Tươi mát, ngọt ngào, ấm áp hay một phong cách nào khác?


Khách hàng: Mình muốn tìm một trai nước hoa, hương gỗ thể hiện sự nam tính. Nhưng mình không muốn quá nặng mùi

Chào bạn! Tuyệt vời! Hương gỗ nam tính nhưng không quá nồng là một lựa chọn rất phổ biến và dễ dùng. Để mình giúp bạn tìm được một mùi hương phù hợp nhất, bạn có thể cho mình biết thêm một chút thông tin được không?

Ví dụ, bạn thích những tông gỗ nào hơn? Gỗ tuyết tùng mạnh mẽ, gỗ đàn hương ấm áp hay gỗ vetiver tươi mát? Hoặc bạn có thích một chút hương thơm nào khác đi kèm với gỗ không? Chẳng hạn như một chút cam chanh tươi sáng, hoặc một chút gia vị cay nồng?


Khách hàng: Mình muốn có một chút gió biển

Tuyệt vời! Hương gỗ kết hợp với gió biển là một sự lựa chọn rất sảng khoái và nam tính. Nó mang lại cảm giác mạnh mẽ nhưng vẫn rất dễ chị

## Phân tích sở thích, nhu cầu nước hoa của khách hàng


In [31]:
full_text_history = "\n".join(history)
history_chat = full_text_history.split("######")[1]

print(history_chat)


ChatBot: Chào bạn, mình là trợ lý ảo của Namperfume. Rất vui được hỗ trợ bạn tìm kiếm mùi hương ưng ý. Bạn có thể cho mình biết bạn đang tìm kiếm loại nước hoa như thế nào không?

Khách hàng: Tôi muốn được tư vấn nước hoa

ChatBot:

Khách hàng: Mình muốn tìm một trai nước hoa, hương gỗ thể hiện sự nam tính. Nhưng mình không muốn quá nặng mùi
Chào bạn! Tuyệt vời! Hương gỗ nam tính nhưng không quá nồng là một lựa chọn rất phổ biến và dễ dùng. Để mình giúp bạn tìm được một mùi hương phù hợp nhất, bạn có thể cho mình biết thêm một chút thông tin được không?

Ví dụ, bạn thích những tông gỗ nào hơn? Gỗ tuyết tùng mạnh mẽ, gỗ đàn hương ấm áp hay gỗ vetiver tươi mát? Hoặc bạn có thích một chút hương thơm nào khác đi kèm với gỗ không? Chẳng hạn như một chút cam chanh tươi sáng, hoặc một chút gia vị cay nồng?

Khách hàng: Mình muốn có một chút gió biển
Tuyệt vời! Hương gỗ kết hợp với gió biển là một sự lựa chọn rất sảng khoái và nam tính. Nó mang lại cảm giác mạnh mẽ nhưng vẫn rất dễ chịu, khôn

# Query Transformations

Dựa vào những sở thích, nhu cầu của khách hàng, hãy tạo ra 5 câu mô tả sản phẩm nước hoa phù hợp với khach hàng:

## Tạo ra 5 câu mô tả khác nhau về sản phẩm cho là phù hợp với khách hàng

In [27]:
query_explore_customer_preferces_template = """
Dựa vào lịch sử trò chuyện sau đây, bạn hãy đóng vai trò là một chuyên gia tư vấn nước hoa cao cấp. Nhiệm vụ của bạn là tạo ra **năm mô tả chi tiết và đa dạng** về sở thích, nhu cầu và mong muốn của khách hàng, nhằm mục đích tìm kiếm các mẫu nước hoa phù hợp nhất từ cơ sở dữ liệu vector.

**Mục tiêu:** Bằng cách khai thác các khía cạnh khác nhau trong yêu cầu của khách hàng, bạn sẽ giúp hệ thống vượt qua những hạn chế của phương pháp tìm kiếm tương tự dựa trên khoảng cách đơn thuần, từ đó mang đến kết quả chính xác và phù hợp hơn.

**Lịch sử trò chuyện:**
{history_chat}

**Yêu cầu:**

* Mỗi mô tả cần tập trung vào việc phân tích và làm rõ các yếu tố sau:
    * Giới tính của khách hàng: Nam/Nữ
    * Mùi hương yêu thích: (ví dụ: tươi mát, hoa cỏ, gỗ, trái cây, vani,...)
    * Mục đích sử dụng: (ví dụ: hàng ngày, dịp đặc biệt, đi làm, hẹn hò,...)
    * Phong cách cá nhân: (ví dụ: năng động, thanh lịch, lãng mạn, cá tính, quyến rũ,...)
    * Ngân sách dự kiến: (ví dụ: 1.000.000 đồng - 2.000.000 đồng, 1.500.000 đồng - 2.000.000 đồng, ...)
* Sử dụng ngôn ngữ chuyên nghiệp, tinh tế và giàu hình ảnh để truyền tải chính xác cảm xúc và mong muốn của khách hàng.
* Tạo ra các mô tả có góc nhìn khác nhau, nhấn mạnh vào các khía cạnh khác nhau trong sở thích của khách hàng.
* Đảm bảo mỗi mô tả đều đủ chi tiết để hệ thống có thể hiểu rõ và tìm kiếm hiệu quả trong cơ sở dữ liệu vector.
* Mô tả cần đúng định dạng đầu ra.

**Định dạng đầu ra:**

Mô tả 1: [Mô tả chi tiết với góc nhìn 1]

Mô tả 2: [Mô tả chi tiết với góc nhìn 2]

Mô tả 3: [Mô tả chi tiết với góc nhìn 3]

Mô tả 4: [Mô tả chi tiết với góc nhìn 4]

Mô tả 5: [Mô tả chi tiết với góc nhìn 5]
"""

In [ ]:
prompt_query_explore_customer_preferces = ChatPromptTemplate.from_template(query_explore_customer_preferces_template)

# Xây dựng chain
chain = (
    prompt_query_explore_customer_preferces 
    | llm 
    | parser
)

response = chain.invoke(history_chat=history_chat)
print(response)

Mô tả 1: Một quý ông hiện đại đang tìm kiếm sự tự tin và năng lượng cho ngày làm việc. Anh ấy muốn một mùi hương nam tính nhưng không áp đảo, một làn gió biển hòa quyện với sự ấm áp của gỗ, tạo nên sự cân bằng hoàn hảo giữa sự sảng khoái và sự trưởng thành. Anh ấy mong muốn một mùi hương giá cả phải chăng (1-2 triệu VNĐ) có thể trở thành dấu ấn cá nhân hàng ngày, một sự khẳng định tinh tế về phong cách và bản lĩnh. Giới tính: Nam, Mùi hương yêu thích: Gỗ, Biển, Mục đích sử dụng: Hàng ngày, Đi làm, Phong cách cá nhân: Năng động, Tự tin, Ngân sách: 1.000.000 - 2.000.000 VNĐ.

Mô tả 2: Người đàn ông này khao khát một mùi hương gợi nhớ đến những buổi sáng trên bờ biển, nơi sự mạnh mẽ của đại dương gặp gỡ sự vững chãi của rừng cây. Anh ta không muốn một loại nước hoa quá phô trương, mà là một sự nam tính tinh tế, gần gũi. Anh ta cần một mùi hương giúp anh ta cảm thấy tràn đầy sinh lực và sẵn sàng đối mặt với mọi thử thách trong công việc, nhưng cũng đủ dễ chịu để không làm phiền đồng nghiệp

In [33]:
def split_descriptions_re(text):
  
    descriptions = re.split(r'\n\nMô tả \d+:\s*', text)
    # Xử lý đoạn đầu tiên nếu nó không bắt đầu bằng "Mô tả"
    final_descriptions = []
    for idx,item in enumerate(descriptions):
        if item.startswith("Mô tả"):
            final_descriptions.append(item)
        else:
            final_descriptions.append(f"Mô tả {idx+1}: " + item)
    return final_descriptions


descriptions = split_descriptions_re(response)
descriptions

['Mô tả 1: Một quý ông hiện đại đang tìm kiếm sự tự tin và năng lượng cho ngày làm việc. Anh ấy muốn một mùi hương nam tính nhưng không áp đảo, một làn gió biển hòa quyện với sự ấm áp của gỗ, tạo nên sự cân bằng hoàn hảo giữa sự sảng khoái và sự trưởng thành. Anh ấy mong muốn một mùi hương giá cả phải chăng (1-2 triệu VNĐ) có thể trở thành dấu ấn cá nhân hàng ngày, một sự khẳng định tinh tế về phong cách và bản lĩnh. Giới tính: Nam, Mùi hương yêu thích: Gỗ, Biển, Mục đích sử dụng: Hàng ngày, Đi làm, Phong cách cá nhân: Năng động, Tự tin, Ngân sách: 1.000.000 - 2.000.000 VNĐ.',
 'Mô tả 2: Người đàn ông này khao khát một mùi hương gợi nhớ đến những buổi sáng trên bờ biển, nơi sự mạnh mẽ của đại dương gặp gỡ sự vững chãi của rừng cây. Anh ta không muốn một loại nước hoa quá phô trương, mà là một sự nam tính tinh tế, gần gũi. Anh ta cần một mùi hương giúp anh ta cảm thấy tràn đầy sinh lực và sẵn sàng đối mặt với mọi thử thách trong công việc, nhưng cũng đủ dễ chịu để không làm phiền đồng n

## Từ 5 câu mô tả trên, thực hiện retrival thông tin các loại nước hoa phù hợp

In [34]:
docs = []
for description in  descriptions:
    docs.append(retriever(description))
docs

[{'ids': [['10952115', '10950786', '10951679']],
  'embeddings': None,
  'documents': [[None, None, None]],
  'uris': None,
  'data': None,
  'metadatas': [[{'brand': 'Prada',
     'compare_at_price': 2430000,
     'count_rate': 46,
     'count_rate_1': 0,
     'count_rate_2': 0,
     'count_rate_3': 0,
     'count_rate_4': 4,
     'count_rate_5': 96,
     'description': 'Hương đầu: Hoa oải hương, Quả cam. Hương giữa: Bạc hà, Xô thơm. Hương cuối: Hạt vông vang (Ambrette – Musk Mallow), Ambroxan. Sự sạch sẽ, gọn gàng và thơm tho của người đàn ông cũng được xem là một nét quyến rũ với người khác giới. Nếu không quá cầu kỳ và điệu đà, một quý ông đôi khi chỉ cần sự thoang thoảng hương thơm như vừa bước ra từ nhà tắm, một cách thơm thanh lịch và nhẹ nhàng. Đó cũng chính là những gì mà Prada Luna Rossa đem tới cho các anh em. Một hương Oải hương nhẹ nhẹ thoang thoảng mở đầu, nhẹ nhàng và mềm mại, không quá mạnh, không ngợp, xen lẫn với sự tươi mát và mọng nước của Quả cam. Sự tươi mát của P

# Re-ranking

In [ ]:
temp = docs[0]['metadatas'][0][0]

In [79]:
def product_description(id,temp):
    brand = temp["brand"]
    name = temp["name"]
    description = temp["description"]
    origin = temp["origin"]
    product_style = temp["product_style"]
    product_note = temp["product_note"]
    product_gender = temp["product_gender"]
    price = temp["price"]
    return f"""\n------------\nID Sản phẩm : {id}\nTên sản phẩm: {name} \nThương hiệu: {brand}\nSản xuất: {origin}\nNước hoa cho: {product_gender} \nGiá bán: {price}\nNhóm hương: {product_note}\nPhong cách: {product_style}\nMô tả: {description}"""
    

In [80]:
product_recomment = []
for doc in docs:
    ids = doc['ids']
    for id, temp in zip(ids[0], doc['metadatas'][0]):
        product =  product_description(id,temp)
        product_recomment.append(product)

In [85]:
prompt_ranking = f"""
Dưới đây là danh sách các sản phẩm nước hoa gợi ý. Nhiệm vụ của bạn là chọn ra 3 sản phầm phù hợp nhất với những thông tin thu thập từ khách hàng

###########
DƯỚI ĐÂY LÀ 5 MÔ TẢ (5 MÔ TẢ NÀY ĐƯỢC TẠO RA TỪ THÔNG TIN THU THẬP TỪ 1 KHÁCH HÀNG) SẢN PHẨM CỦA MỘT KHÁCH HÀNG VỀ NHU CẦU MUA NƯỚC HOA

{response}

##########
DANH SÁCH SẢN PHẨM 

"""

for item in product_recomment:
    prompt_ranking += item
    
print(prompt_ranking)


Dưới đây là danh sách các sản phẩm nước hoa gợi ý. Nhiệm vụ của bạn là chọn ra 3 sản phầm phù hợp nhất với những thông tin thu thập từ khách hàng

###########
DƯỚI ĐÂY LÀ 5 MÔ TẢ (5 MÔ TẢ NÀY ĐƯỢC TẠO RA TỪ THÔNG TIN THU THẬP TỪ 1 KHÁCH HÀNG) SẢN PHẨM CỦA MỘT KHÁCH HÀNG VỀ NHU CẦU MUA NƯỚC HOA

Mô tả 1: Một quý ông hiện đại đang tìm kiếm sự tự tin và năng lượng cho ngày làm việc. Anh ấy muốn một mùi hương nam tính nhưng không áp đảo, một làn gió biển hòa quyện với sự ấm áp của gỗ, tạo nên sự cân bằng hoàn hảo giữa sự sảng khoái và sự trưởng thành. Anh ấy mong muốn một mùi hương giá cả phải chăng (1-2 triệu VNĐ) có thể trở thành dấu ấn cá nhân hàng ngày, một sự khẳng định tinh tế về phong cách và bản lĩnh. Giới tính: Nam, Mùi hương yêu thích: Gỗ, Biển, Mục đích sử dụng: Hàng ngày, Đi làm, Phong cách cá nhân: Năng động, Tự tin, Ngân sách: 1.000.000 - 2.000.000 VNĐ.

Mô tả 2: Người đàn ông này khao khát một mùi hương gợi nhớ đến những buổi sáng trên bờ biển, nơi sự mạnh mẽ của đại dương 

In [84]:
print(llm.generate_content(prompt_ranking).text)

Dựa trên thông tin thu thập từ khách hàng (cả 5 mô tả đều tương đồng), đây là 3 sản phẩm phù hợp nhất:

1.  **Mr. Burberry Indigo (ID Sản phẩm: 27395052):**
    *   **Giá:** 1.350.000 VNĐ (Phù hợp ngân sách)
    *   **Nhóm hương:** Bạc hà, Hoa hương thảo, Quả chanh vàng, Hoa tím (Có hương biển - bạc hà, và hương gỗ - rêu sồi, hổ phách)
    *   **Phong cách:** Lịch lãm, Nam tính, Cuốn hút (Phù hợp với môi trường công sở, sự tự tin)
    *   **Mô tả:** Mở đầu tươi mát, nam tính, không quá ồn ào, nhẹ nhàng.

2.  **Versace Pour Homme Dylan Blue (ID Sản phẩm: 10951424):**
    *   **Giá:** 1.880.000 VNĐ (Phù hợp ngân sách)
    *   **Nhóm hương:** Hương ambroxan, Cam bergamot, Quả bưởi, Hương nước (Hương biển và gỗ)
    *   **Phong cách:** Nam tính, Tươi mát, Hấp dẫn (Phù hợp với phong cách năng động, trẻ trung)
    *   **Mô tả:** Mở đầu tinh khiết, trong trẻo của hương nước, cam Bergamot, bưởi. Nam tính của hương Ambroxan, gai góc của tiêu đen.

3.  **Prada Luna Rossa (ID Sản phẩm: 10952115):

#  Đề xuất loại nước hoa - thu thập phản hồi của khách hàng

## Nếu khách hàng chôt sản phẩm thì thực hiện chốt đơn

## Nếu khách hàng không ưng ý thì lập lại quy trình dựa trên phản hồi của khách hàng để gợi ý sản phẩm khác.

# Gửi maill chốt đơn.